# Demo UI

In [ ]:
!pip install openai cassandra-driver llama-index

In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
import openai
from llama_index import ListIndex
from llama_index.readers.schema.base import Document
from IPython.display import Markdown, display

In [3]:
# keys and tokens here
openai_api_key = "<open_api_key>"
openai.api_key = openai_api_key
cass_user = '<user>'
cass_pw = '<pwd>'
scb_path = 'secure-connect-vector-search-demo.zip'

In [4]:
model_id = "text-embedding-ada-002"

In [ ]:
cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider(cass_user, cass_pw)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace('workspan')

In [6]:
def get_queries_across_opportunities():
    # This function returns a list of queries.
    # You can modify this to fetch the options from a different source if required.
    queries = ['Identify the wins',
               'Identify opportunities with next step to schedule a meeting',
               'I want to know more about the customer and the challenges']
    return queries

In [7]:
def get_queries_specific_to_opportunity():
    # This function returns a list of queries.
    # You can modify this to fetch the options from a different source if required.
    queries = ['What is the sentiment on this opportunity?',
               'What are the next steps for this opportunity?',
               'What are the open items for this opportunity?',
               'What are the challenges for this opportunity?']
    return queries

In [8]:
def query_opportunity_sentiment(opportunity):
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = '{opportunity}' ;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        results.append(row.sentiment)
    return results[0]

In [36]:
import re

def query_opportunity_nextsteps(opportunity):
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = '{opportunity}' ;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        results.append(row.llm_output)

    next_steps = ["Next Steps section not found!"]
    # Find the section for "Next Steps"
    match = re.search(r'Next steps to schedule a meeting:(.*?)(?=Challenges:|Open Items:|$)', results[0], re.DOTALL | re.IGNORECASE)

    if match:
        next_steps_section = match.group(1).strip()
        next_steps = [step.strip('- ').strip() for step in next_steps_section.split('\n') if step]

    next_steps_str = " ".join([step if step.endswith('.') else step + '.' for step in next_steps])

    return next_steps_str


In [25]:
def query_opportunity_openitems(opportunity):
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = '{opportunity}' ;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        results.append(row.llm_output)

    open_items = ["Open Items section not found!"]
    # Find the section for "Open Items"
    match = re.search(r'Open Items:(.*?)(?=Challenges:|Next Steps:|$)', results[0], re.DOTALL | re.IGNORECASE)

    if match:
        open_items_section = match.group(1).strip()
        open_items = [item.strip('- ').strip() for item in open_items_section.split('\n') if item]

    open_items_str = " ".join([item if item.endswith('.') else item + '.' for item in open_items])

    return open_items_str

In [26]:
def query_opportunity_challenges(opportunity):
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and opportunity_id = '{opportunity}' ;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        results.append(row.llm_output)

    challenges = ["Challenges section not found!"]
    # Find the section for "Challenges"
    match = re.search(r'Challenges:(.*?)(?=Next Steps:|Open Items:|$)', results[0], re.DOTALL | re.IGNORECASE)

    if match:
        challenges_section = match.group(1).strip()
        challenges = [challenge.strip('- ').strip() for challenge in challenges_section.split('\n') if challenge]

    challenges_str = " ".join([challenge if challenge.endswith('.') else challenge + '.' for challenge in challenges])

    return challenges_str

In [27]:
def get_opportunities():
    # This function returns a list of oppertunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT opportunity_id FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS'  ;'''
    rows = session.execute(cqlSelect)
    opportunities = []
    for row_i, row in enumerate(rows):
        opportunities.append(row.opportunity_id)
    return opportunities

In [28]:
def query_customer_wins():
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and sentiment = 'positive'  ;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        results.append(row.opportunity_id)

    results_str = " ".join([result if result.endswith('.') else result + '.' for result in results])

    return results_str

In [29]:
def query_customer_next_step_to_schedule_meeting():
    # This function returns a list of opportunities.
    # You can modify this to fetch the options from a different source if required.
    vectorsearchon = 'next action to set up a meeting'
    embedding = openai.Embedding.create(input= vectorsearchon, model=model_id)['data'][0]['embedding']
    cqlSelect = f'''SELECT * FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' and llm_output : 'schedule a meeting' ORDER BY llm_output_embedding ANN OF {embedding} LIMIT 10;'''
    rows = session.execute(cqlSelect)
    results = []
    for row_i, row in enumerate(rows):
        next_steps = ["Next Steps section not found!"]
        # Find the section for "Next Steps"
        match = re.search(r'Next steps to schedule a meeting:(.*?)(?=Challenges:|Open Items:|$)', row.llm_output, re.DOTALL | re.IGNORECASE)

        if match:
            next_steps_section = match.group(1).strip()
            next_steps = [step.strip('- ').strip() for step in next_steps_section.split('\n') if step]

        next_steps_str = " ".join([step if step.endswith('.') else step + '.' for step in next_steps])
        results.append(row.opportunity_id + ' : ' + next_steps_str + '\n' + '\n')

    results_str = " ".join([result if result.endswith('.') else result + '.' for result in results])

    return results_str

In [30]:
def query_customer_challenges():
    vectorsearchon = 'find opportunity with listed challenges'
    embedding = openai.Embedding.create(input= vectorsearchon, model=model_id)['data'][0]['embedding']

    cqlSelect = f'''SELECT llm_output FROM workspan.customer_opportunities WHERE customer_id = 'CUS100' and partner_id = 'AWS' ORDER BY llm_output_embedding ANN OF {embedding} LIMIT 10;  '''
    rows = session.execute(cqlSelect)
    #print(rows)
    documents = []
    for item in rows:
        documents.append(Document(text=str(item)))
        #print(str(item))

    index = ListIndex.from_documents(documents)

    # set Logging to DEBUG for more detailed outputs
    query_engine = index.as_query_engine()
    response = query_engine.query("What are the Challenges?")

    return f"{response}"

In [31]:
# Function to be called on button click
def on_button_click(button):
    opportunity_value = opportunity.value
    query_value = query.value
    if query_value == "What is the sentiment on this opportunity?":
        result.value = query_opportunity_sentiment(opportunity.value)
    elif query_value == "What are the next steps for this opportunity?":
        result.value = query_opportunity_nextsteps(opportunity.value)
    elif query_value == "What are the open items for this opportunity?":
        result.value = query_opportunity_openitems(opportunity.value)
    elif query_value == "What are the challenges for this opportunity?":
        result.value = query_opportunity_challenges(opportunity.value)

In [32]:
# Function to be called on button click
def on_cus_button_click(button):
    query_value = cus_query.value
    if query_value == "Identify the wins":
        cus_result.value = query_customer_wins()
    elif query_value == "Identify opportunities with next step to schedule a meeting":
        cus_result.value = query_customer_next_step_to_schedule_meeting()
    elif query_value == "I want to know more about the customer and the challenges":
        cus_result.value = query_customer_challenges()

In [34]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import Image as IPyImage

# 1. Get the image in bytes
with open("workspan_logo.png", "rb") as f:
    image_data = f.read()

# Create an Image widget from the image data
logo = widgets.Image(value=image_data, format='png', width=100, height=100)

# Create a Label widget for the title
title = widgets.Label(value="Workspan Demo - Customer Name: Teradyne, Inc.", layout=widgets.Layout(margin='0 0 0 10px'))

# Create dropdown and button as before
opportunity = widgets.Dropdown(options=get_opportunities(),  description='Select Opportunity:', layout=widgets.Layout(width='250px', margin='0px 10px 0 0'), style={'description_width': 'initial'})
query = widgets.Dropdown(options=get_queries_specific_to_opportunity(),  description='Select Query:', layout=widgets.Layout(width='350px', margin='0px 10px 0 0'), style={'description_width': 'initial'})
button = widgets.Button(description="Submit", layout=widgets.Layout(width='100px'))
result = widgets.Textarea(value='',
                          placeholder='',
                          description='Result:',
                          disabled=True,
                          layout=widgets.Layout(width='610px', margin='10px 0 0 0', justify_content='flex-end'),
                          rows=7,
                          style={'description_width': '18%'})

# New line or spacer
spacer = widgets.HTML(value="<br>")


# custer specific queries
cus_query = widgets.Dropdown(options=get_queries_across_opportunities(),  description='Queries across opportunities:', layout=widgets.Layout(width='610px', margin='0px 10px 0 0'), style={'description_width': 'initial'})
cus_button = widgets.Button(description="Submit", layout=widgets.Layout(width='100px'))
cus_result = widgets.Textarea(value='',
                          placeholder='',
                          description='Result:',
                          disabled=True,
                          layout=widgets.Layout(width='610px', margin='10px 0 0 0', justify_content='flex-end'),
                          rows=20,
                          style={'description_width': '18%'})

# Organize logo and title horizontally, then everything vertically
header = widgets.HBox([logo, title])
#layout = widgets.VBox([header, spacer, widgets.HBox([opportunity, query, button]), result])

layout = widgets.VBox([
    header,
    spacer,
    widgets.HBox([opportunity, query, button]),
    result,
    spacer,
    spacer,
    widgets.HBox([cus_query, cus_button]),  # Organizing custom query and button horizontally
    cus_result
])

# Display the layout
display(layout)

# Bind button click
button.on_click(on_button_click)

cus_button.on_click(on_cus_button_click)
